In [2]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

### Finding Giant Component

In [25]:
from pathlib import Path

path = Path('../data/processed_except/relation_avg.csv').resolve();
di_graph = nx.DiGraph()
with open(path, 'r') as input_file:
    while True:
        line = input_file.readline().rstrip()

        # EOF
        if line == '':
            break

        splits = line.split(',')
        # prevent self-loop
        if splits[0] != splits[1]:
            di_graph.add_edge(splits[1], splits[0])

# Finding the Giant Component
undi_graph = di_graph.to_undirected()
graph_components = (undi_graph.subgraph(c) for c in nx.connected_components(undi_graph));
graph_components = sorted(graph_components, key=len, reverse=True)

giant_component =  di_graph.subgraph(graph_components[0].nodes())

edge_df = pd.DataFrame(columns=[ 'source','dest' ])
for index, edge in enumerate(giant_component.edges()):
    (src, dest) = edge
    edge_df.loc[index] = [ src, dest ]

edge_df.head()

,source,dest
0,80,52
1,80,4
2,80,1
3,80,81
4,80,31


### Degree Centrality

In [10]:
from operator import itemgetter
degree_centreality = sorted(nx.degree_centrality(giant_component).items(), key=itemgetter(1), reverse=True)
deg_df = pd.DataFrame(columns=["tag", "degree centreality"])
for index, deg in enumerate(degree_centreality):
    deg_df.loc[index] = [ deg[0], deg[1] ]
deg_df.head(10)

,tag,degree centreality
0,4,1.120921
1,1,0.955854
2,7,0.852207
3,8,0.719770
4,6,0.500960
5,26,0.458733
6,84,0.443378
7,47,0.443378
8,52,0.410749
9,29,0.314779


### Edge Betweenness Centrality
The edge betweenness centrality is defined as the number of the shortest paths that go through an edge in a graph or network, Each edge in the network can be associated with an edge betweenness centrality value. An edge with a high edge betweenness centrality score represents a bridge-like connector between two parts of a network, and the removal of which may affect the communication between many pairs of nodes through the shortest paths between them. The removal of this edge will result in a partition of the network into two densely connected subnetworks. 

In [15]:
from operator import itemgetter
edge_betweeness = sorted(nx.edge_betweenness_centrality(giant_component).items(), key=itemgetter(1), reverse=True)
ebet_df = pd.DataFrame(columns=["source", "dest", "edge betweeness"])
for index, bet in enumerate(edge_betweeness):
    ebet_df.loc[index] = [bet[0][0], bet[0][1], bet[1]]
ebet_df.head(10)

,source,dest,edge betweeness
0,1,7,0.011826
1,7,1,0.009957
2,1,4,0.006238
3,4,142,0.005123
4,4,1,0.004442
5,7,84,0.003831
6,1,142,0.003518
7,84,7,0.003484
8,1,26,0.003458
9,1,84,0.003348


In [35]:
modified_giant_component_ebet =  nx.DiGraph()
modified_giant_component_ebet.add_edges_from(giant_component.edges())
modified_giant_component_ebet.remove_edge('1', '7')
modified_giant_component_ebet.remove_edge('7', '1')

from operator import itemgetter
edge_betweeness = sorted(nx.edge_betweenness_centrality(modified_giant_component_ebet).items(), key=itemgetter(1), reverse=True)
ebet_df = pd.DataFrame(columns=["source", "dest", "edge betweeness"])
for index, bet in enumerate(edge_betweeness):
    ebet_df.loc[index] = [bet[0][0], bet[0][1], bet[1]]

,source,dest,edge betweeness
0,1,4,0.008006
1,4,1,0.005790
2,4,142,0.005174
3,7,84,0.004166
4,1,26,0.004106
5,84,7,0.003678
6,7,4,0.003602
7,1,6,0.003481
8,1,84,0.003460
9,9,1,0.003455


In [37]:
ebet_df.tail(10)

,source,dest,edge betweeness
5769,1663,1697,0.000004
5770,140,143,0.000004
5771,235,443,0.000004
5772,552,574,0.000004
5773,335,144,0.000004
5774,562,176,0.000004
5775,1260,1257,0.000004
5776,1260,1258,0.000004
5777,113,293,0.000004
5778,244,460,0.000004


### Betweenness Centrality
In graph theory, betweenness centrality is a measure of centrality in a graph based on shortest paths. For every pair of vertices in a connected graph, there exists at least one shortest path between the vertices such that either the number of edges that the path passes through (for unweighted graphs) or the sum of the weights of the edges (for weighted graphs) is minimized. The betweenness centrality for each vertex is the number of these shortest paths that pass through the vertex

In [12]:
from operator import itemgetter
betweenness_centrality = sorted(nx.betweenness_centrality(giant_component).items(), key=itemgetter(1), reverse=True)
betweenness_centrality
bet_df = pd.DataFrame(columns=["tag", "betweenness centrality"])
for index, bet in enumerate(betweenness_centrality):
    bet_df.loc[index] = [ bet[0], bet[1] ]
bet_df.head(10)

,tag,betweenness centrality
0,1,0.229033
1,4,0.198131
2,7,0.152898
3,84,0.060731
4,8,0.054278
5,6,0.030691
6,26,0.028589
7,24,0.023541
8,29,0.022166
9,9,0.021237


### Eigenvector Centrality
In graph theory, eigenvector centrality (also called eigencentrality) is a measure of the **influence of a node** in a network. Relative scores are assigned to all nodes in the network based on the concept that connections to high-scoring nodes contribute more to the score of the node in question than equal connections to low-scoring nodes. A high eigenvector score means that a node is connected to many nodes who themselves have high scores.

In [13]:
from operator import itemgetter
eigenvector_centrality = sorted(nx.eigenvector_centrality(giant_component).items(), key=itemgetter(1), reverse=True)
eig_df = pd.DataFrame(columns=["tag", "eigenvector centrality"])
for index, eig in enumerate(eigenvector_centrality):
    eig_df.loc[index] = [ eig[0], eig[1] ]
eig_df.head(10)

,tag,eigenvector centrality
0,4,0.276747
1,8,0.231120
2,7,0.222649
3,1,0.218088
4,6,0.191541
5,47,0.182304
6,26,0.177157
7,52,0.176597
8,84,0.142907
9,12,0.141635


In [36]:
comm = list(nx.algorithms.community.k_clique_communities(giant_component.to_undirected(), 3))
print(len(comm))

1
